<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [19]</a>'.</span>

In [1]:
import os 
import sys 
import pandas as pd 
import numpy as np 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
import pdb

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [3]:
%load_ext autoreload
%autoreload 2

## Set up 

In [4]:
with open('data/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [5]:
os.environ.update(credentials)

In [6]:
scope = "user-read-recently-played"

In [7]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

## Functions

In [8]:
def split_utc_time_str(time_str):
    part1 = datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) - datetime.timedelta(hours=4)
    date = str(part1.date())
    time = str(part1.time().strftime('%H:%M'))
    return(date, time)

In [9]:
def get_recently_played(after=None):
    recently_played = spotify.current_user_recently_played(after=after)
    # Describe json 
    print('Getting songs at ', datetime.datetime.now())
    before = recently_played['cursors']['before']
    after = recently_played['cursors']['after']
    n_items = len(recently_played['items'])
    print(f'There are {n_items} songs between '
          , datetime.datetime.fromtimestamp(int(before)/1000)
          , '  and  '
          , datetime.datetime.fromtimestamp(int(after)/1000))
    
    if n_items == 0:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    # Parse recently played output 
    songs_list = []
    wanted_keys = ['name', 'duration_ms', 'popularity', 'uri']
    for song in recently_played['items']:
        temp_song_dict = {}
        temp_song_dict['after_ts'] = after
        temp_song_dict['played_at_date'], temp_song_dict['played_at_time'] = split_utc_time_str(song['played_at'])
        for k in wanted_keys:
            temp_song_dict[k] = song['track'][k]
            
        # Take first artist information 
        temp_song_dict['artist_name'] = song['track']['artists'][0]['name'] 
        temp_song_dict['artist_id'] = song['track']['artists'][0]['uri']

        if song['context'] != None:
            temp_song_dict['playlist_id'] = song['context']['uri']
        else:
            temp_song_dict['playlist_id'] = None
        songs_list.append(temp_song_dict)

    
    # Export cleanly 
    out_series = pd.DataFrame(songs_list)
    out_series['duration_min'] = out_series['duration_ms']/60000.0
    out_series = out_series.rename({'uri':'song_uri'}, axis=1) 
    out_series = out_series[['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id', 'after_ts']]
    
    return(out_series)

## Get time of last Spotify Pull

In [10]:
db_location = 'data/listening_history.db'

In [11]:
# Parameters
db_location = "data/listening_history.db"


In [12]:
con = sqlite3.connect(db_location)

In [13]:
cursor = con.cursor()

In [14]:
rslt = cursor.execute('select max(after_ts) from Listening_History').fetchone()
latest_time_pull = rslt[0]
print("Latest pull from ", 
datetime.datetime.fromtimestamp(int(latest_time_pull)/1000)) 

Latest pull from  2020-08-02 09:50:53.875000


## Pull from Spotify and Store to DB 

In [15]:
newly_played = get_recently_played(latest_time_pull)

Getting songs at  2020-08-02 10:28:35.933009
There are 2 songs between  2020-08-02 10:24:05.329000   and   2020-08-02 10:26:51.733000


In [16]:
newly_played.head()

,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,that's just how it goes,ROLE MODEL,2020-08-02,10:26,2.773333,60,spotify:track:4J4rJEaVpgr2tHwX6Jd13H,spotify:artist:1dy5WNgIKQU6ezkpZs4y8z,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1596378411733
1,Amor De Una Noche (feat. Julio Voltio) - Remix...,N'Klabe,2020-08-02,10:24,4.455767,53,spotify:track:6B9r2hH46YbSEvnivmeF5C,spotify:artist:3sXLxfvWccL0EI4Q3xxO3q,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1596378411733


In [17]:
con = sqlite3.connect(db_location)
newly_played.to_sql('Listening_History', con, if_exists='append' )

In [18]:
con.commit()
con.close()

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [19]:
raise ValueError()

ValueError: 